In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests
import numpy as np
# Carregando as variáveis de ambiente do arquivo .env
load_dotenv()

True

In [2]:
ano = "2022"

In [3]:
from data_master_eng_ml.utils.twitch_api import build_query,fetch_data_with_pagination

2024-08-21 22:21:59.447 | INFO     | data_master_eng_ml.config:<module>:11 - PROJ_ROOT path is: /Users/josepaulo/data_master_eng_ml


In [4]:
URL_TWITCH_BASE = "https://api.igdb.com/v4"

In [5]:
# Fighting - 4
# Racing - 10
# 6	Full Release

In [6]:
from data_master_eng_ml.utils.mappings import region_mapping_inverted,plataform_mapping,player_perspectives_mapping,genres_mapping,game_modes_mapping

Token expirado ou inexistente. Obtendo um novo...
Novo token obtido com sucesso: {'access_token': '9h7pd499bsg7e9zli5biuddfl5sq01', 'expires_in': 5179563, 'token_type': 'bearer'}


In [7]:
# Exemplo de uso
url = f"{URL_TWITCH_BASE}/release_dates"

# Define os campos a serem selecionados
fields = [
    "category",
    "created_at",
    "date",
    "game",
    "human",
    "platform",
    "region",
    "status",
    "updated_at",
    "y",
]

# Define os filtros a serem aplicados
filters = {"status": "= 6", "y": f"= {ano}", "region": f"= 8"}

# Chamada da função com paginação
data_frame_games_find = fetch_data_with_pagination(url, build_query, fields, filters)
data_frame_games_find["region_name"] = data_frame_games_find["region"].map(
    region_mapping_inverted
)


In [8]:
games_id = list(set(data_frame_games_find["game"].to_list()))

In [9]:
games_id_list = ",".join(map(str, games_id))

In [10]:
len(games_id_list)

1110072

In [11]:
# Exemplo de uso
url = f"{URL_TWITCH_BASE}/involved_companies"

# Define os campos a serem selecionados
fields = ["*"]
# Define os filtros a serem aplicados
filters = {
    "game": f"= ({games_id_list})",
}

# Chamada da função com paginação
data_frame_companies_find = fetch_data_with_pagination(
    url, build_query, fields, filters
)
data_frame_companies_find
company_id = list(set(data_frame_companies_find["company"].to_list()))
company_id_list = ",".join(map(str, company_id))

Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax Error",
    "status": 400
  }
]
Erro ao obter dados: 400 - [
  {
    "title": "Syntax 

KeyboardInterrupt: 

In [ ]:
data_frame_companies_find

,id,company,created_at,developer,game,porting,publisher,supporting,updated_at,checksum
0,42850,3951,1481500800,True,14016,False,False,False,1481587200,4019ec1f-ed4f-42e2-e693-f40326782648
1,257101,22440,1709880472,True,227913,False,True,False,1710125858,c611d8b7-c58c-114f-d621-30f32ff9ae85
2,258679,53624,1710720793,True,219363,False,True,False,1710722577,aec26cb7-5a2c-a090-d8c9-f4e25f3714ae
3,71561,16916,1542153600,True,105594,False,True,False,1542153600,54ce12d4-13cd-f897-3327-659d3193c279
4,273902,18505,1719758406,False,147323,False,True,False,1719758482,448933cf-1b64-2ac6-a164-d364162105da
...,...,...,...,...,...,...,...,...,...,...
2073,273388,43575,1719577332,False,153890,False,True,False,1719794891,f9a38275-0639-9b64-afe7-09ac7bdc025d
2074,273389,47681,1719577332,True,153890,False,False,False,1719794891,aa125d57-56ef-c89b-af52-0efac48e4ee5
2075,273629,2104,1719627940,True,305861,False,True,False,1720810639,7dda16dc-e11b-c971-6756-c2a6314e480a
2076,274592,55986,1720097141,True,307501,False,True,False,1721258535,96446d91-b392-01d2-fbb5-9d22a40b4b23


In [ ]:
import pycountry_convert as pc
import pycountry


def country_to_continent(country_code):
    try:
        # Converte o código do país para o nome do continente
        country = pycountry.countries.get(numeric=str(country_code))
        continent_code = pc.country_alpha2_to_continent_code(country.alpha_2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except:
        return "Unknown"  # Retorna 'Unknown' se o código do país não for reconhecido

In [ ]:
def array_count(value):
    if value == "Unknown":
        return -1
    else:
        return len(value)

In [ ]:
# Exemplo de uso
url = f"{URL_TWITCH_BASE}/companies"

# Define os campos a serem selecionados
fields = [
    "developed",
    "slug",
    "published",
    "country",
    "start_date",
    "start_date_category",
    "parent",
]
# Define os filtros a serem aplicados
filters = {
    "id": f"= ({company_id_list})",
}
# Chamada da função com paginação
data_frame_companies = fetch_data_with_pagination(url, build_query, fields)
data_frame_companies["developed"] = data_frame_companies["developed"].fillna("Unknown")
data_frame_companies["published"] = data_frame_companies["published"].fillna("Unknown")
data_frame_companies["country"] = data_frame_companies["country"].fillna(1)
data_frame_companies["country"] = data_frame_companies["country"].astype(int)
data_frame_companies["games_developed"] = data_frame_companies["developed"].apply(
    array_count
)
data_frame_companies["has_parents"] = data_frame_companies["parent"].notna().astype(int)
data_frame_companies["games_published"] = data_frame_companies["published"].apply(
    array_count
)
data_frame_companies["continent_name"] = data_frame_companies["country"].apply(
    country_to_continent
)

data_frame_companies = data_frame_companies.drop(
    columns=["developed", "published", "country", "parent", "start_date_category"]
)

In [ ]:
# Inner Join (default)
companies_join = pd.merge(
    data_frame_companies_find,
    data_frame_companies,
    left_on="company",
    right_on="id",
    how="inner",
)

In [ ]:
df_companies_final = companies_join[
    ["game", "games_developed", "has_parents", "games_published", "continent_name"]
]

In [ ]:
# Exemplo de uso
url = f"{URL_TWITCH_BASE}/multiplayer_modes"

# Define os campos a serem selecionados
fields = [
    "campaigncoop",
    "game",
    "lancoop",
    "offlinecoop",
    "offlinecoopmax",
    "offlinemax",
    "onlinecoop",
    "onlinecoopmax",
    "onlinemax",
    "splitscreen",
]
# Define os filtros a serem aplicados
filters = {
    "game": f"= ({games_id_list})",
}

# Chamada da função com paginação
data_frame_multiplayer_modes = fetch_data_with_pagination(
    url, build_query, fields, filters
)
# Converte todas as colunas booleanas em 0/1
boolean_columns = data_frame_multiplayer_modes.select_dtypes(include='bool').columns
data_frame_multiplayer_modes[boolean_columns] = data_frame_multiplayer_modes[boolean_columns].astype(int)

data_frame_multiplayer_modes

,id,campaigncoop,game,lancoop,offlinecoop,offlinemax,onlinecoop,splitscreen,offlinecoopmax,onlinecoopmax,onlinemax
0,11345,0,95083,0,0,2.0,0,0,NaN,NaN,NaN
1,9771,0,122666,0,0,NaN,0,0,NaN,NaN,NaN
2,5456,0,13205,0,0,0.0,1,0,0.0,4.0,0.0
3,8032,0,34894,0,0,0.0,1,0,0.0,0.0,16.0
4,9819,0,116599,0,0,NaN,1,0,NaN,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
182,28661,0,231049,0,1,4.0,0,1,4.0,NaN,NaN
183,28772,0,40971,0,0,2.0,0,0,NaN,NaN,NaN
184,28771,0,40971,0,0,2.0,0,0,NaN,NaN,NaN
185,28847,0,307264,0,0,2.0,0,0,NaN,NaN,2.0


In [ ]:
# Exemplo de uso
url = f"{URL_TWITCH_BASE}/games"

# Define os campos a serem selecionados
fields = [
    "name",
    "game_modes",
    "genres",
    "age_ratings",
    "involved_companies",
    "player_perspectives",
    "platforms",
    "rating",
    "remasters",
]

# Define os filtros a serem aplicados
filters = {
    "id": f"= ({games_id_list})",
    "category": f"= (0)",
}

# Chamada da função com paginação
data_frame_games = fetch_data_with_pagination(url, build_query, fields, filters)

# Trata as colunas
data_frame_games["player_perspectives"] = data_frame_games[
    "player_perspectives"
].fillna("Unknown")
data_frame_games["game_modes"] = data_frame_games["game_modes"].fillna("unknown_game_mode")
data_frame_games["genres"] = data_frame_games["genres"].fillna("unknown_genres_name")
data_frame_games["player_perspective_name"] = (
    data_frame_games["player_perspectives"]
    .map(lambda x: [player_perspectives_mapping.get(i, "unknown_player_perspectives") for i in x])
    .apply(lambda x: list(set(x)))
)
data_frame_games["platforms_name"] = (
    data_frame_games["platforms"]
    .map(lambda x: [plataform_mapping.get(i, "unknown_platforms_name") for i in x])
    .apply(lambda x: list(set(x)))
)
data_frame_games["genres_first"] = (
    data_frame_games["genres"]
    .map(lambda x: [genres_mapping.get(i, "unknown_genres_name") for i in x])
    .apply(lambda x: list(set(x))).apply(lambda x: x[0])
)
data_frame_games["game_modes_name"] = (
    data_frame_games["game_modes"]
    .map(lambda x: [data_frame_game_modes_mapping.get(i, "unknown_game_mode") for i in x])
    .apply(lambda x: list(set(x)))
)
data_frame_games["has_remaster"] = data_frame_games["remasters"].notna()
data_frame_games["target"] = np.where(data_frame_games["rating"].isna(),0,1)
data_frame_games = data_frame_games.drop(
    columns=[
        "game_modes",
        "player_perspectives",
        "remasters",
        "genres",
        "platforms",
        "involved_companies",
        "rating",
        
    ]
)


Erro ao obter dados: 413 - Content Too Large


KeyError: 'player_perspectives'

In [ ]:
data_frame_games

,id,age_ratings,name,player_perspective_name,platforms_name,genres_first,game_modes_name,has_remaster,target
0,71,"[984, 4395, 47678, 47679, 47680, 47681, 47682]",Portal,[first-person],"[modern_console, mobile, pc]",shooter,[single-player],False,1
1,72,"[11721, 32022, 47684, 47685, 47687, 125644, 14...",Portal 2,[first-person],"[modern_console, pc]",shooter,"[multiplayer, single-player, co-operative, spl...",False,1
2,1344,"[526, 49158]",Wild Guns,[third-person],"[classic_console, unknown_platforms_name, mode...",arcade,"[co-operative, single-player, split-screen]",True,1
3,4250,"[65229, 142924, 179089]",Kingdom: The Far Reaches,[third-person],"[less_common_portable_console, unknown_platfor...",adventure,[single-player],False,0
4,6614,"[8996, 32717, 99650]",Zero Escape: Virtue's Last Reward,"[first-person, text]","[modern_console, portable_console, pc]",visual-novel,[single-player],False,1
...,...,...,...,...,...,...,...,...,...
1713,117988,"[38375, 53582, 95803, 162474, 162475, 162477, ...",Gnosia,"[side-view, text]","[modern_console, portable_console, pc]",visual-novel,[single-player],False,1
1714,63346,"[38299, 58796, 98015, 119664, 159757, 159758, ...",The Sushi Spinnery,[bird-view-slash-isometric],"[mobile, modern_console, pc]",strategy,[single-player],False,0
1715,158682,"[163668, 163669]",Cave Digger 2: Dig Harder,"[virtual-reality, first-person]","[modern_console, unknown_platforms_name, vr, pc]",adventure,"[multiplayer, single-player, co-operative]",False,0
1716,153890,"[94295, 95386, 97445, 125572, 131331, 140471]",Unusual Findings,"[side-view, text]","[modern_console, pc]",adventure,[single-player],False,1


In [ ]:
age_rating_mapping = {
    1: "3+",
    2: "6+",
    3: "12+",
    4: "16+",
    5: "18+",
    6: "Rating Pending",
    7: "All Ages",
    8: "All Ages",
    9: "10+",
    10: "12+",
    11: "18+",
    12: "18+",
    13: "All Ages",
    14: "12+",
    15: "16+",
    16: "18+",
    17: "18+",
    18: "All Ages",
    19: "6+",
    20: "12+",
    21: "16+",
    22: "18+",
    23: "All Ages",
    24: "12+",
    25: "15+",
    26: "18+",
    27: "18+",
    28: "All Ages",
    29: "10+",
    30: "12+",
    31: "14+",
    32: "16+",
    33: "18+",
    34: "All Ages",
    35: "6+",
    36: "15+",
    37: "15+",
    38: "18+",
    39: "18+",
}

In [ ]:
# Definir uma ordem para os grupos etários
age_order = pd.CategoricalDtype(
    categories=[
        "Rating Pending",
        "All Ages",
        "3+",
        "6+",
        "10+",
        "12+",
        "14+",
        "15+",
        "16+",
        "18+",
    ],
    ordered=True,
)

In [ ]:
def age_classif(list):
    url = f"{URL_TWITCH_BASE}/age_ratings"

    try:
        lista_query = ",".join(map(str, list))
        # Define os campos a serem selecionados
        fields = ["rating"]

        filters = {
            "id": f"= ({lista_query})",
        }
        # Chamada da função com paginação
        data_frame = fetch_data_with_pagination(url, build_query, fields, filters)
        # # Exemplo de uso no DataFrame
        data_frame["age_rating_group"] = data_frame["rating"].map(age_rating_mapping)
        # Convertendo a coluna 'age_rating_group' para essa ordem
        data_frame["age_rating_group"] = data_frame["age_rating_group"].astype(
            age_order
        )
        # Pegar o maior valor de 'age_rating_group'
        max_age_rating_group = data_frame["age_rating_group"].max()
        return max_age_rating_group
    except:
        return "No Rating"

In [ ]:
data_frame_games["age_classif"] = data_frame_games["age_ratings"].apply(age_classif).drop(columns=["age_ratings",])

In [ ]:
data_frame_games =  data_frame_games.drop(columns=["age_ratings"])


In [ ]:
data_frame_games

,id,name,player_perspective_name,platforms_name,genres_first,game_modes_name,has_remaster,target,age_classif
0,71,Portal,[first-person],"[modern_console, mobile, pc]",shooter,[single-player],False,1,15+
1,72,Portal 2,[first-person],"[modern_console, pc]",shooter,"[multiplayer, single-player, co-operative, spl...",False,1,12+
2,1344,Wild Guns,[third-person],"[classic_console, unknown_platforms_name, mode...",arcade,"[co-operative, single-player, split-screen]",True,1,12+
3,4250,Kingdom: The Far Reaches,[third-person],"[less_common_portable_console, unknown_platfor...",adventure,[single-player],False,0,12+
4,6614,Zero Escape: Virtue's Last Reward,"[first-person, text]","[modern_console, portable_console, pc]",visual-novel,[single-player],False,1,18+
...,...,...,...,...,...,...,...,...,...
1713,117988,Gnosia,"[side-view, text]","[modern_console, portable_console, pc]",visual-novel,[single-player],False,1,15+
1714,63346,The Sushi Spinnery,[bird-view-slash-isometric],"[mobile, modern_console, pc]",strategy,[single-player],False,0,3+
1715,158682,Cave Digger 2: Dig Harder,"[virtual-reality, first-person]","[modern_console, unknown_platforms_name, vr, pc]",adventure,"[multiplayer, single-player, co-operative]",False,0,18+
1716,153890,Unusual Findings,"[side-view, text]","[modern_console, pc]",adventure,[single-player],False,1,15+


In [ ]:
data_frame_games_find[["game", "region_name",]]

df_companies_final
data_frame_multiplayer_modes

,id,campaigncoop,game,lancoop,offlinecoop,offlinemax,onlinecoop,splitscreen,offlinecoopmax,onlinecoopmax,onlinemax
0,11345,0,95083,0,0,2.0,0,0,NaN,NaN,NaN
1,9771,0,122666,0,0,NaN,0,0,NaN,NaN,NaN
2,5456,0,13205,0,0,0.0,1,0,0.0,4.0,0.0
3,8032,0,34894,0,0,0.0,1,0,0.0,0.0,16.0
4,9819,0,116599,0,0,NaN,1,0,NaN,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
182,28661,0,231049,0,1,4.0,0,1,4.0,NaN,NaN
183,28772,0,40971,0,0,2.0,0,0,NaN,NaN,NaN
184,28771,0,40971,0,0,2.0,0,0,NaN,NaN,NaN
185,28847,0,307264,0,0,2.0,0,0,NaN,NaN,2.0


In [ ]:
data_frame_games_find.drop_duplicates(subset="game")

,id,category,created_at,date,game,human,platform,region,updated_at,y,status,region_name
0,594406,0,1715767847,1645574400,139339,"Feb 23, 2022",34,8,1715907478,2022,6,worldwide
3,594525,0,1715836304,1652832000,200839,"May 18, 2022",48,8,1715916873,2022,6,worldwide
4,594526,0,1715836308,1663804800,219337,"Sep 22, 2022",6,8,1715916888,2022,6,worldwide
9,594868,0,1715924700,1649289600,301306,"Apr 07, 2022",6,8,1715931145,2022,6,worldwide
10,594710,0,1715900186,1647388800,301158,"Mar 16, 2022",24,8,1715973541,2022,6,worldwide
...,...,...,...,...,...,...,...,...,...,...,...,...
3740,588874,0,1714166425,1659916800,118734,"Aug 08, 2022",6,8,1714673946,2022,6,worldwide
3741,588981,0,1714184209,1665100800,219627,"Oct 07, 2022",6,8,1714681791,2022,6,worldwide
3742,616762,0,1720796077,1672272000,291577,"Dec 29, 2022",6,5,1721896239,2022,6,japan
3744,590645,0,1714717197,1654819200,194704,"Jun 10, 2022",6,8,1714717236,2022,6,worldwide


In [ ]:
df_companies_games = pd.merge(
    df_companies_final,
    data_frame_games_find[["game", "region_name",]].drop_duplicates(subset="game"),
    on="game",
    how="inner",
)
df_companies_games

,game,games_developed,has_parents,games_published,continent_name,region_name
0,14016,22,0,-1,Unknown,worldwide
1,227913,11,0,3,Unknown,worldwide
2,219363,1,0,1,Unknown,worldwide
3,105594,1,0,1,Unknown,worldwide
4,147323,1,0,28,Unknown,worldwide
...,...,...,...,...,...,...
2073,153890,-1,0,11,Asia,worldwide
2074,153890,3,0,1,Unknown,worldwide
2075,305861,57,0,67,North America,worldwide
2076,307501,1,0,1,Unknown,worldwide


In [ ]:
df_companies_games_multiplayer = pd.merge(
    df_companies_games.drop_duplicates(subset="game"),
    data_frame_multiplayer_modes[
        ["game", "onlinecoop", "onlinecoopmax", "onlinemax", "splitscreen"]
    ],
    on="game",
    how="left",
)
df_companies_games_multiplayer

,game,games_developed,has_parents,games_published,continent_name,region_name,onlinecoop,onlinecoopmax,onlinemax,splitscreen
0,14016,22,0,-1,Unknown,worldwide,NaN,NaN,NaN,NaN
1,227913,11,0,3,Unknown,worldwide,NaN,NaN,NaN,NaN
2,219363,1,0,1,Unknown,worldwide,NaN,NaN,NaN,NaN
3,105594,1,0,1,Unknown,worldwide,1.0,NaN,NaN,0.0
4,147323,1,0,28,Unknown,worldwide,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1453,264077,6,0,-1,Unknown,worldwide,NaN,NaN,NaN,NaN
1454,153890,-1,0,11,Asia,worldwide,NaN,NaN,NaN,NaN
1455,305861,57,0,67,North America,worldwide,NaN,NaN,NaN,NaN
1456,307501,1,0,1,Unknown,worldwide,NaN,NaN,NaN,NaN


In [ ]:
data_frame_multiplayer_modes[
    ["game", "onlinecoop", "onlinecoopmax", "onlinemax", "splitscreen"]
]

,game,onlinecoop,onlinecoopmax,onlinemax,splitscreen
0,95083,0,NaN,NaN,0
1,122666,0,NaN,NaN,0
2,13205,1,4.0,0.0,0
3,34894,1,0.0,16.0,0
4,116599,1,2.0,2.0,0
...,...,...,...,...,...
182,231049,0,NaN,NaN,1
183,40971,0,NaN,NaN,0
184,40971,0,NaN,NaN,0
185,307264,0,NaN,2.0,0


In [ ]:
df_join = pd.merge(
    data_frame_games,
    df_companies_games_multiplayer,
    left_on="id",
    right_on="game",
    how="left",
).drop(columns=['game']).drop_duplicates(subset="id")

In [ ]:
# Explode as listas de 'attributes' para transformar cada elemento em uma linha separada
df_exploded = df_join[['id',"platforms_name"]].explode('platforms_name')

# Converte a coluna 'attributes' em colunas de dummies, com prefixo para evitar conflitos
dummies = pd.get_dummies(df_exploded['platforms_name'].str.replace('-', '_'))

# Agrupa por 'id' e faz a soma para agrupar as flags 0/1
dummies = dummies.groupby(df_exploded['id']).sum().reset_index()

# Junte os dummies ao DataFrame original
df_join = df_join.drop(columns=['platforms_name']).merge(dummies, on='id')


In [ ]:
# Explode as listas de 'attributes' para transformar cada elemento em uma linha separada
df_exploded = df_join[['id',"game_modes_name"]].explode('game_modes_name')

# Converte a coluna 'attributes' em colunas de dummies, com prefixo para evitar conflitos
dummies = pd.get_dummies(df_exploded['game_modes_name'].str.replace('-', '_'))

# Agrupa por 'id' e faz a soma para agrupar as flags 0/1
dummies = dummies.groupby(df_exploded['id']).sum().reset_index()

# Junte os dummies ao DataFrame original
df_join = df_join.drop(columns=['game_modes_name']).merge(dummies, on='id')


In [ ]:
# Explode as listas de 'attributes' para transformar cada elemento em uma linha separada
df_exploded = df_join[['id',"player_perspective_name"]].explode('player_perspective_name')

# Converte a coluna 'attributes' em colunas de dummies, com prefixo para evitar conflitos
dummies = pd.get_dummies(df_exploded['player_perspective_name'].str.replace('-', '_'))

# Agrupa por 'id' e faz a soma para agrupar as flags 0/1
dummies = dummies.groupby(df_exploded['id']).sum().reset_index()

# Junte os dummies ao DataFrame original
df_join = df_join.drop(columns=['player_perspective_name']).merge(dummies, on='id')


In [ ]:
# Converte todas as colunas booleanas em 0/1
boolean_columns = df_join.select_dtypes(include='bool').columns
df_join[boolean_columns] = df_join[boolean_columns].astype(int)


In [ ]:
import numpy as np

In [ ]:
df_join['has_global_launch'] = np.where(df_join['region_name'] == 'worldwide',1,0)

In [ ]:
df_join = df_join.drop(columns=['region_name'])

In [ ]:
df_join

,id,name,genres_first,has_remaster,target,age_classif,games_developed,has_parents,games_published,continent_name,...,unknown_game_mode,auditory,bird_view_slash_isometric,first_person,side_view,text,third_person,unknown_player_perspectives,virtual_reality,has_global_launch
0,71,Portal,shooter,0,1,15+,125.0,0.0,1346.0,North America,...,0,0,0,1,0,0,0,0,0,1
1,72,Portal 2,shooter,0,1,12+,86.0,0.0,94.0,North America,...,0,0,0,1,0,0,0,0,0,1
2,1344,Wild Guns,arcade,1,1,12+,12.0,0.0,36.0,Europe,...,0,0,0,0,0,0,1,0,0,0
3,4250,Kingdom: The Far Reaches,adventure,0,0,12+,4.0,0.0,39.0,Unknown,...,0,0,0,0,0,0,1,0,0,1
4,6614,Zero Escape: Virtue's Last Reward,visual-novel,0,1,18+,1.0,0.0,132.0,Europe,...,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,225446,Dropship Commander,arcade,0,0,All Ages,3.0,0.0,3.0,North America,...,0,0,0,0,1,0,0,0,1,1
1301,250264,Tokyo Pinball,arcade,0,0,No Rating,1.0,0.0,1.0,Unknown,...,0,0,1,0,0,0,0,0,0,1
1302,307160,Tanuki Tales,arcade,0,0,No Rating,1.0,0.0,-1.0,Unknown,...,0,0,0,0,0,0,1,0,0,1
1303,229164,Cubic Light,indie,0,0,3+,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
df_join.to_csv(f'twitch_api_data_{ano}.csv',header=True,index=False)